# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

In [10]:
import pandas as pd
import re

In [5]:
data = pd.read_csv('../data/energy_task.csv')
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,11-01-2016 17:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,18.2,48.900000,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3
1,11-01-2016 17:10,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,18.2,48.863333,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2
2,11-01-2016 17:20,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,18.2,48.730000,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1
3,11-01-2016 17:30,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,18.1,48.590000,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0
4,11-01-2016 17:40,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,18.1,48.590000,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9



### Выполним предобработку данных

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19626 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19569 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19654 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19644 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19651 non-null  float64
 15  T7           19681 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19613 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

#### Заполним пропуски

In [7]:
data['date'] = data['date'].fillna(data['date'].mode())
data = data.fillna({'RH_2':data['RH_2'].mean(), 'RH_4':data['RH_4'].mean(), 'RH_5':data['RH_5'].mean(), 'RH_6':data['RH_6'].mean(), 'T7':data['T7'].mean(), 'T8':data['T8'].mean(), 'T9':data['T9'].mean(), 'RH_9':data['RH_9'].mean(), 'T_out':data['T_out'].mean(), 'Press_mm_hg':data['Press_mm_hg'].mean(), 'RH_out':data['RH_out'].mean(), 'Visibility':data['Visibility'].mean()})

In [8]:
print(*data['date'].mode())

01-02-2016 17:10 01-03-2016 00:50 01-04-2016 11:00 01-05-2016 00:40 01-05-2016 13:30 01-05-2016 20:10 02-04-2016 03:00 02-04-2016 08:30 02-05-2016 06:50 03-02-2016 18:10 03-02-2016 23:10 03-05-2016 01:30 04-04-2016 03:00 04-05-2016 18:00 05-02-2016 11:20 05-03-2016 10:30 05-03-2016 16:40 05-04-2016 21:30 05-05-2016 05:50 05-05-2016 20:30 06-02-2016 01:40 06-03-2016 23:20 07-02-2016 03:20 07-02-2016 13:20 07-03-2016 02:40 07-04-2016 01:10 07-04-2016 07:40 07-05-2016 03:50 07-05-2016 15:10 08-02-2016 19:30 08-05-2016 02:10 08-05-2016 20:30 09-03-2016 20:00 09-05-2016 00:10 09-05-2016 15:40 10-03-2016 14:30 10-03-2016 20:50 10-04-2016 17:30 11-02-2016 00:30 11-02-2016 14:20 11-03-2016 23:30 11-04-2016 02:50 11-04-2016 06:40 11-04-2016 23:20 12-01-2016 10:20 12-02-2016 08:20 12-02-2016 09:00 12-03-2016 10:50 12-05-2016 06:00 12-05-2016 19:50 12-05-2016 22:50 13-02-2016 13:40 13-03-2016 07:30 13-04-2016 12:50 13-05-2016 13:20 13-05-2016 23:30 14-05-2016 00:50 15-02-2016 10:10 15-02-2016 10:

In [9]:
print(pd.isnull(data).sum())

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
dtype: int64


#### Преобразуем категориальные признаки в числовые

In [28]:
d = pd.DataFrame()
d[['date', 'time']] = data['date'].str.split(' ', expand= True)
d

,date,time
0,11-01-2016,17:00
1,11-01-2016,17:10
2,11-01-2016,17:20
3,11-01-2016,17:30
4,11-01-2016,17:40
...,...,...
19730,27-05-2016,17:20
19731,27-05-2016,17:30
19732,27-05-2016,17:40
19733,27-05-2016,17:50


In [14]:
d[['day', 'mth', 'year']] = list(map(int, d['date'].str.split('-', expand= True)))
d[['hour', 'min']] = list(map(int, d['time'].str.split(':', expand= True)))
d.drop(['date', 'time'], axis= 1 , inplace= True )
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   day     19735 non-null  int64
 1   mth     19735 non-null  int64
 2   year    19735 non-null  int64
 3   hour    19735 non-null  int64
 4   min     19735 non-null  int64
dtypes: int64(5)
memory usage: 771.0 KB


In [16]:
data.drop(['date'], axis= 1 , inplace= True )
data2 = pd.concat([
    d,
    data
], axis=1)

In [17]:
data2

,day,mth,year,hour,min,Appliances,lights,T1,RH_1,T2,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,0,1,2,0,1,60,30,19.890000,47.596667,19.200000,...,18.2000,48.900000,17.033333,45.5300,6.60,733.5,92.000000,7.000000,63.000000,5.3
1,0,1,2,0,1,60,30,19.890000,46.693333,19.200000,...,18.2000,48.863333,17.066667,45.5600,6.48,733.6,92.000000,6.666667,59.166667,5.2
2,0,1,2,0,1,50,30,19.890000,46.300000,19.200000,...,18.2000,48.730000,17.000000,45.5000,6.37,733.7,92.000000,6.333333,55.333333,5.1
3,0,1,2,0,1,50,40,19.890000,46.066667,19.200000,...,18.1000,48.590000,17.000000,45.4000,6.25,733.8,92.000000,6.000000,51.500000,5.0
4,0,1,2,0,1,60,40,19.890000,46.333333,19.200000,...,18.1000,48.590000,17.000000,45.4000,6.13,733.9,92.000000,5.666667,47.666667,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,0,1,2,0,1,100,0,25.566667,46.560000,25.890000,...,24.7000,50.074000,23.200000,46.7900,22.70,755.2,55.666667,3.333333,23.666667,13.3
19731,0,1,2,0,1,90,0,25.500000,46.500000,25.754000,...,24.7000,49.790000,23.200000,46.7900,22.60,755.2,56.000000,3.500000,24.500000,13.3
19732,0,1,2,0,1,270,10,25.500000,46.596667,25.628571,...,24.7000,49.660000,23.200000,46.7900,22.50,755.2,56.333333,3.666667,25.333333,13.3
19733,0,1,2,0,1,420,10,25.500000,46.990000,25.414000,...,24.6625,49.518750,23.200000,46.8175,22.30,755.2,56.666667,3.833333,26.166667,13.2


In [18]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          19735 non-null  int64  
 1   mth          19735 non-null  int64  
 2   year         19735 non-null  int64  
 3   hour         19735 non-null  int64  
 4   min          19735 non-null  int64  
 5   Appliances   19735 non-null  int64  
 6   lights       19735 non-null  int64  
 7   T1           19735 non-null  float64
 8   RH_1         19735 non-null  float64
 9   T2           19735 non-null  float64
 10  RH_2         19735 non-null  float64
 11  T3           19735 non-null  float64
 12  RH_3         19735 non-null  float64
 13  T4           19735 non-null  float64
 14  RH_4         19735 non-null  float64
 15  T5           19735 non-null  float64
 16  RH_5         19735 non-null  float64
 17  T6           19735 non-null  float64
 18  RH_6         19735 non-null  float64
 19  T7  

In [19]:
data2.to_csv('../data/energy_task_new2.csv')

### Выделение целевого признака и предикторов

In [2]:
data = pd.read_csv('../data/energy_task_new2.csv')
data.head()

,Unnamed: 0,day,mth,year,hour,min,Appliances,lights,T1,RH_1,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,0,0,1,2,0,1,60,30,19.89,47.596667,...,18.2,48.900000,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3
1,1,0,1,2,0,1,60,30,19.89,46.693333,...,18.2,48.863333,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2
2,2,0,1,2,0,1,50,30,19.89,46.300000,...,18.2,48.730000,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1
3,3,0,1,2,0,1,50,40,19.89,46.066667,...,18.1,48.590000,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0
4,4,0,1,2,0,1,60,40,19.89,46.333333,...,18.1,48.590000,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9


In [3]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data

,day,mth,year,hour,min,Appliances,lights,T1,RH_1,T2,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,0,1,2,0,1,60,30,19.890000,47.596667,19.200000,...,18.2000,48.900000,17.033333,45.5300,6.60,733.5,92.000000,7.000000,63.000000,5.3
1,0,1,2,0,1,60,30,19.890000,46.693333,19.200000,...,18.2000,48.863333,17.066667,45.5600,6.48,733.6,92.000000,6.666667,59.166667,5.2
2,0,1,2,0,1,50,30,19.890000,46.300000,19.200000,...,18.2000,48.730000,17.000000,45.5000,6.37,733.7,92.000000,6.333333,55.333333,5.1
3,0,1,2,0,1,50,40,19.890000,46.066667,19.200000,...,18.1000,48.590000,17.000000,45.4000,6.25,733.8,92.000000,6.000000,51.500000,5.0
4,0,1,2,0,1,60,40,19.890000,46.333333,19.200000,...,18.1000,48.590000,17.000000,45.4000,6.13,733.9,92.000000,5.666667,47.666667,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,0,1,2,0,1,100,0,25.566667,46.560000,25.890000,...,24.7000,50.074000,23.200000,46.7900,22.70,755.2,55.666667,3.333333,23.666667,13.3
19731,0,1,2,0,1,90,0,25.500000,46.500000,25.754000,...,24.7000,49.790000,23.200000,46.7900,22.60,755.2,56.000000,3.500000,24.500000,13.3
19732,0,1,2,0,1,270,10,25.500000,46.596667,25.628571,...,24.7000,49.660000,23.200000,46.7900,22.50,755.2,56.333333,3.666667,25.333333,13.3
19733,0,1,2,0,1,420,10,25.500000,46.990000,25.414000,...,24.6625,49.518750,23.200000,46.8175,22.30,755.2,56.666667,3.833333,26.166667,13.2


In [4]:
y = data['Appliances']
X = data.drop(['Appliances'], axis=1)

In [45]:
y.head()

0    60
1    60
2    50
3    50
4    60
Name: Appliances, dtype: int64

In [46]:
X.head()

,day,mth,year,hour,min,lights,T1,RH_1,T2,RH_2,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,0,1,2,0,1,30,19.89,47.596667,19.2,44.790000,...,18.2,48.900000,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3
1,0,1,2,0,1,30,19.89,46.693333,19.2,44.722500,...,18.2,48.863333,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2
2,0,1,2,0,1,30,19.89,46.300000,19.2,44.626667,...,18.2,48.730000,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1
3,0,1,2,0,1,40,19.89,46.066667,19.2,44.590000,...,18.1,48.590000,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0
4,0,1,2,0,1,40,19.89,46.333333,19.2,44.530000,...,18.1,48.590000,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9


### Разделение данных на обучающую и тестовую выборки

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# указав параметр test_size = 0.2 получим, что 20% данных уйдут в тестовую выборку
# также в функцию можно передать параметр shuffle: по умолчанию функция рандомно
# перемешивает данные; поэтому и модели будут получаться разные;
# задайте параметр shuffle = False, если не хотите перемешивать

In [48]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15788, 30), (15788,), (3947, 30), (3947,))

### Линейная регрессия в sklearn

In [6]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [50]:
lr = LinearRegression().fit(X_train, y_train)

In [51]:
lr_pred = lr.predict(X_test)
lr_pred

array([148.46792764, 112.49374429,  80.7753418 , ..., 102.56751099,
        93.41305382,  99.15719128])

In [52]:
lr.coef_

array([ 1.31901572e-15,  8.95394869e-14,  1.77191595e-13, -3.44169138e-14,
        0.00000000e+00,  2.05793319e+00, -8.11344783e-01,  1.50850106e+01,
       -1.60214793e+01, -1.27155441e+01,  2.51303671e+01,  4.39163617e+00,
       -3.46405492e+00, -5.34422932e-01, -8.50740392e-01,  1.59171232e-01,
        6.07118200e+00,  2.27159525e-01,  1.21450091e+00, -1.50376842e+00,
        6.81859984e+00, -4.91267950e+00, -1.42718106e+01, -5.22825319e-01,
       -5.20231375e+00,  1.56967948e-01, -1.43100734e-01,  1.94140967e+00,
        1.44517885e-01,  5.81116660e-01])

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

In [54]:
print(f'MAE: {mean_absolute_error(y_test, lr_pred)}')
print(f'MSE: {mean_squared_error(y_test, lr_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, lr_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, lr_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')

MAE: 52.48189958193038
MSE: 8461.841986625426
RMSE: 91.98827091877217
MAPE: 0.7919221452049117
R^2: 0.15753163691522865


### Применим регулязацию

#### Ridge

In [8]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np

In [56]:
# для поиска гиперпараметров мы всегда сначала формируем словарь
# ключами словаря являются названия гиперпараметров
# значениями - список (или массив numpy) с возможными значениями
# параметры описываются в документации к моделям, так что можно и нужно гуглить
parameters = {'alpha': np.arange(0.1, 1, 0.1)}

In [57]:
# а теперь пробуем подобрать значение этого гиперпараметра с помощью GridSearchCV
ridge = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
ridge_pred

array([148.46703891, 112.49148658,  80.7770838 , ..., 102.57531458,
        93.4127739 ,  99.15713873])

In [59]:
r = ridge.best_estimator_
r.coef_

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   2.05800979,  -0.81270989,  15.08303744,
       -16.0151447 , -12.71262526,  25.12643139,   4.39146482,
        -3.46511626,  -0.53496253,  -0.85192647,   0.1591664 ,
         6.07009735,   0.22713118,   1.21288989,  -1.50382808,
         6.81750408,  -4.91241047, -14.26663945,  -0.52290179,
        -5.20121193,   0.15695488,  -0.14310707,   1.94143771,
         0.14451163,   0.58057936])

In [60]:
print(f'MAE: {mean_absolute_error(y_test, ridge_pred)}')
print(f'MSE: {mean_squared_error(y_test, ridge_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, ridge_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, ridge_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')


MAE: 52.48162102260653
MSE: 8461.828305928542
RMSE: 91.98819655764832
MAPE: 0.7919198415890041
R^2: 0.15753299897733375


#### Lasso

In [61]:
lasso = GridSearchCV(Lasso(), parameters).fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)
lasso_pred

array([147.53385451, 110.27041398,  81.51837409, ..., 104.63149155,
        94.773134  ,  99.57775215])

In [62]:
l = lasso.best_estimator_
l.coef_

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   2.06710713,  -0.        ,  14.73844051,
       -14.97155278, -12.09764083,  23.70536794,   3.97355086,
        -3.4215071 ,  -0.43971221,  -0.85685504,   0.14985293,
         5.58375529,   0.20304221,   0.        ,  -1.4727062 ,
         6.02964687,  -4.90975225, -12.43125346,  -0.40011439,
        -4.39024273,   0.12109283,  -0.08386522,   1.85388941,
         0.14202729,  -0.        ])

In [63]:
print(f'MAE: {mean_absolute_error(y_test, lasso_pred)}')
print(f'MSE: {mean_squared_error(y_test, lasso_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, lasso_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, lasso_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')

MAE: 52.39560492224129
MSE: 8464.93805977989
RMSE: 92.00509800972928
MAPE: 0.7910877606378934
R^2: 0.1572233891735958


### Полиномиальная регрессия

In [9]:
from sklearn.preprocessing import PolynomialFeatures
# создаем объект, который позволит расширить множество предикторов
pf = PolynomialFeatures(2)  
# добавляем новые предикторы
X_p=pf.fit_transform(X_train) 
X_p

array([[1.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        8.41000000e+02, 2.66800000e+02, 8.46400000e+01],
       [1.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        1.02400000e+03, 3.68000000e+01, 1.32250000e+00],
       [1.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        1.60000000e+03, 1.48000000e+02, 1.36900000e+01],
       ...,
       [1.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        1.60000000e+03, 1.80000000e+02, 2.02500000e+01],
       [1.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        3.84400000e+03, 5.92100000e+02, 9.12025000e+01],
       [1.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        7.11111111e+02, 1.94666667e+02, 5.32900000e+01]])

In [65]:
X_p_test=pf.fit_transform(X_test) 
X_p_test

array([[ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         1.60000000e+03,  1.12000000e+02,  7.84000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         1.60000000e+03,  6.40000000e+01,  2.56000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         5.13777778e+02,  7.54800000e+00,  1.10889000e-01],
       ...,
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         1.60000000e+03,  1.40000000e+02,  1.22500000e+01],
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         8.50694445e+02,  2.52875000e+02,  7.51689000e+01],
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         3.86469444e+03, -1.38010000e+02,  4.92840000e+00]])

In [66]:
lr_p = LinearRegression().fit(X_p, y_train)
lr_p_pred = lr_p.predict(X_p_test)
lr_p_pred

array([153.71498108,  71.08558655,  74.7559967 , ...,  69.66493225,
       138.02137756,  74.99598694])

In [67]:
lr_p.coef_

array([-1.41622788e-08,  1.29437154e+10, -3.33691586e+09,  3.36367370e+10,
        3.30945828e+09, -8.77801191e+09, -2.80311954e+10, -1.57096975e+09,
        2.71784544e+09, -3.15188859e+09,  1.19142456e+10, -8.82536524e+07,
       -1.90173329e+08,  7.78757074e+08,  1.02462736e+09,  3.21507645e+08,
        7.19201296e+07, -8.49078765e+08, -1.28616248e+09, -5.34781202e+09,
       -6.20615072e+08,  4.84575895e+07, -1.24085847e+09, -8.93678838e+08,
       -9.89537414e+08, -2.04989175e+09,  1.93425337e+09,  2.24247406e+09,
       -3.26434569e+08, -9.14750542e+08, -2.57420772e+07,  1.23161643e+06,
       -1.48425600e+08, -1.20507425e+07,  5.79532628e+07, -2.74905217e+07,
       -2.61482779e+07,  8.16556898e+06, -8.48342933e+06,  7.57349919e+06,
       -7.45133853e+06,  2.28647356e+07,  2.51208125e+07, -2.49990224e+07,
       -3.88902230e+07,  7.40624161e+06, -6.50592123e+06, -2.48603414e+06,
        9.41653432e+06, -2.83097379e+07, -3.90110192e+07, -9.96795774e+06,
        3.63808652e+07, -

In [68]:
print(f'MAE: {mean_absolute_error(y_test, lr_p_pred)}')
print(f'MSE: {mean_squared_error(y_test, lr_p_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, lr_p_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, lr_p_pred))}')
print(f'R^2: {lr_p.score(X_p_test, y_test)}')

MAE: 49.61956117145315
MSE: 7176.236694715011
RMSE: 84.71267139404242
MAPE: 0.7653957533205893
R^2: 0.2855276202437733


#### Ridge

In [69]:
ridge_p = GridSearchCV(Ridge(), parameters).fit(X_p, y_train)
ridge_p_pred = ridge_p.predict(X_p_test)
ridge_p_pred

array([149.54613087,  65.06173734,  73.90847526, ...,  75.54866225,
       140.9670401 ,  80.50392174])

In [70]:
r_p = ridge_p.best_estimator_
r_p.coef_

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.99868963e+00,  1.22035048e+02,
        2.43318127e+01, -2.27029768e+01, -7.83141477e+00,  4.62770648e+01,
        3.90427881e+01,  9.94751880e+00, -3.15231775e+01, -2.86989160e+01,
       -3.58247217e+00,  7.13570981e+01,  5.72581050e+00, -2.22120184e+02,
        2.67801785e+01,  1.02829808e+02, -7.26631729e+00,  9.08697267e+01,
       -4.38430457e+00,  4.24552428e+01,  4.60790686e+00,  7.34001431e+00,
       -3.62031103e+01,  1.59126139e-01, -1.26055604e+02,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [71]:
print(f'MAE: {mean_absolute_error(y_test, ridge_p_pred)}')
print(f'MSE: {mean_squared_error(y_test, ridge_p_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, ridge_p_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, ridge_p_pred))}')
print(f'R^2: {ridge_p.score(X_p_test, y_test)}')

MAE: 49.24863249163712
MSE: 7136.25357824695
RMSE: 84.4763492241879
MAPE: 0.7612883908392258
R^2: 0.2895083741665144


#### Lasso

In [72]:
lasso_p = GridSearchCV(Lasso(), parameters).fit(X_p, y_train)

C:\ML\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.160e+07, tolerance: 1.343e+04
  model = cd_fast.enet_coordinate_descent(
C:\ML\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.169e+07, tolerance: 1.354e+04
  model = cd_fast.enet_coordinate_descent(
C:\ML\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.076e+07, tolerance: 1.324e+04
  model = cd_fast.enet_coordi

C:\ML\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.181e+07, tolerance: 1.343e+04
  model = cd_fast.enet_coordinate_descent(
C:\ML\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.193e+07, tolerance: 1.354e+04
  model = cd_fast.enet_coordinate_descent(
C:\ML\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.099e+07, tolerance: 1.324e+04
  model = cd_fast.enet_coordi

In [74]:
lasso_p_pred = lasso_p.predict(X_p_test)
lasso_p_pred

array([162.13553093,  93.18962124,  80.80145754, ...,  90.40079388,
       144.43498139,  76.10878552])

In [75]:
l_p = lasso_p.best_estimator_
l_p.coef_

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  2.23686921e+00,  0.00000000e+00,
        2.84596615e+00,  0.00000000e+00,  0.00000000e+00,  2.07934982e+01,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -9.50516815e-02, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.07738664e-01, -1.21712628e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [76]:
print(f'MAE: {mean_absolute_error(y_test, lasso_p_pred)}')
print(f'MSE: {mean_squared_error(y_test, lasso_p_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, lasso_p_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, lasso_p_pred))}')
print(f'R^2: {lasso_p.score(X_p_test, y_test)}')

MAE: 49.476245321753176
MSE: 7674.570415653236
RMSE: 87.60462553800019
MAPE: 0.7576494805847765
R^2: 0.23591308066570715


In [39]:
def MAE(y_test, y_pred):
    return np.mean(abs(y_test-y_pred))

def MSE(y_test, y_pred):
    return np.mean((y_test - y_pred)**2)

def RMSE(y_test, y_pred):
    return sqrt(MSE(y_test, y_pred))

def MAPE(y_test, y_pred):
    return np.mean(abs((y_test-y_pred)/y_test))

def R2(y_test, y_pred):
    return 1 - (np.sum((y_test - y_pred)**2) / (np.sum((y_test - np.mean(y_pred))**2)))

In [69]:
class LR:
    
    def __init__(self, lr = 0.0001, i = 5000): 
        self.lr = lr 
        self.i = i
        
    def transform(self, x):
        return np.concatenate((np.ones((len(x), 1)), x), axis = 1)
        
    def Loss(self, x, y, w):
        return sum((y-np.dot(x, w))**2)/x.shape[0]
        
    def fit(self, x, y):
        dist = np.inf
        eps = 1e-4
        X = self.transform(x)
        w = np.zeros((X.shape[1]))
        k = 0
        while dist > eps and k <= self.i:
            L = self.Loss(X,y,w)
            w = w - self.lr*2*np.dot(X.T,y -(np.dot(X, w))/X.shape[0])
            dist = np.abs(L - self.Loss(X,y,w))
            k += 1
        self.w = w
        
    def coef(self):
        return self.w
    
    def predict(self, x):
        x = self.transform(x)
        return (np.dot(x, self.w))    

In [70]:
lr = LR()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(y_pred)
print(lr.coef())

[-1.27245541e+156 -1.26443901e+156 -1.25282991e+156 ... -1.26760267e+156
 -1.29044317e+156 -1.27289990e+156]
[-2.11473481e+150  0.00000000e+000 -2.11473481e+150 -4.22946961e+150
  0.00000000e+000 -2.11473481e+150 -8.00423885e+150 -4.58362753e+151
 -8.51856281e+151 -4.29757992e+151 -8.55471769e+151 -4.70547782e+151
 -8.30346130e+151 -4.40746733e+151 -8.25726172e+151 -4.14101033e+151
 -1.07908438e+152 -1.66095748e+151 -1.16231267e+152 -4.28224294e+151
 -7.49099162e+151 -4.65510467e+151 -9.09005111e+151 -4.11744637e+151
 -8.79265371e+151 -1.55844887e+151 -1.59789406e+153 -1.68986684e+152
 -8.51998395e+150 -8.09932019e+151 -7.94882778e+150]


In [85]:
from scipy. special import logsumexp
class RidgeLR:
    
    def __init__(self, lr = 0.0001, i = 5000, a = 0.9): 
        self.lr = lr 
        self.i = i
        self.a = a
        
    def transform(self, x):
        return np.concatenate((np.ones((len(x), 1)), x), axis = 1)
        
    def Loss(self, x, y, w):
        return (sum((y-np.dot(x, w))**2) + (self.a*(np.dot(w.T, w))))/x.shape[0]
        
    def fit(self, x, y):
        dist = np.inf
        eps = 1e-4
        X = self.transform(x)
        w = np.zeros((X.shape[1]))
        k = 0
        while dist > eps and k <= self.i:
            L = self.Loss(X,y,w)
            w = w - (self.lr*2*np.dot(X.T,(y - np.dot(X, w))) + (2*self.a*w))/X.shape[0]
            dist = np.abs(L - self.Loss(X,y,w))
            k += 1
        self.w = w
        
    def coef(self):
        return self.w
    
    def predict(self, x):
        x = self.transform(x)
        return (np.dot(x, self.w))    

In [86]:
lr = RidgeLR()
lr.fit(X_train, y_train)
#y_pred = lr.predict(X_test)
#print(y_pred)
#print(lr.coef())

C:\Users\user\AppData\Local\Temp\ipykernel_11420\529120379.py:24: RuntimeWarning: invalid value encountered in scalar subtract
  dist = np.abs(L - self.Loss(X,y,w))
